# 📚 RAG Basics: Document Q&A with AI Beast

Learn to build Retrieval-Augmented Generation (RAG) applications that answer questions from your documents.

## What You'll Learn

1. Understand RAG architecture
2. Load and chunk documents
3. Create embeddings
4. Query with context retrieval
5. Build a simple Q&A system

## Prerequisites

- Completed previous tutorials
- Qdrant running (`beast up qdrant`)

## 1. Understanding RAG

RAG enhances LLMs by providing relevant context from your documents.

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   Question  │────▶│  Retrieval  │────▶│     LLM     │
└─────────────┘     └──────┬──────┘     └──────┬──────┘
                           │                    │
                    ┌──────▼──────┐      ┌─────▼─────┐
                    │   Vector    │      │  Answer   │
                    │   Store     │      │  + Source │
                    └─────────────┘      └───────────┘
```

**Benefits:**
- Ground responses in your data
- Reduce hallucinations
- Keep knowledge up-to-date
- Maintain source attribution

In [ ]:
import sys
sys.path.insert(0, '..')

from modules.rag import RAGPipeline, Document, ChunkingStrategy
from modules.llm import LLMClient

# Initialize components
llm = LLMClient()
print("LLM client ready!")

## 2. Loading Documents

First, let's load some documents to work with.

In [ ]:
# Create sample documents (in real use, load from files)
sample_docs = [
    Document(
        content="""AI Beast is a local AI infrastructure manager that helps developers
        run AI models on their own hardware. It supports multiple providers including
        Ollama for local models, OpenAI, and Anthropic for cloud models. The system
        uses Docker containers to manage services and provides a unified API for
        interacting with different AI providers.""",
        metadata={"source": "overview.md", "section": "introduction"}
    ),
    Document(
        content="""To install AI Beast, clone the repository and run the bootstrap
        script. The minimum requirements are 16GB RAM and 50GB disk space. For GPU
        acceleration, NVIDIA GPUs with CUDA support are recommended. Apple Silicon
        Macs can use Metal for acceleration through Ollama.""",
        metadata={"source": "installation.md", "section": "requirements"}
    ),
    Document(
        content="""The RAG module allows you to build document Q&A applications.
        It uses Qdrant as the vector database for storing embeddings. Documents
        are chunked, embedded using local models, and stored for semantic search.
        The pipeline supports multiple chunking strategies including fixed-size,
        semantic, and sentence-based chunking.""",
        metadata={"source": "rag_guide.md", "section": "overview"}
    ),
    Document(
        content="""Ollama is a tool for running large language models locally.
        It supports models like Llama 3, Mistral, Phi, and many others. Ollama
        handles model downloading, memory management, and provides a simple API.
        In AI Beast, Ollama is the default local provider for both chat and
        embedding models.""",
        metadata={"source": "ollama_guide.md", "section": "introduction"}
    ),
    Document(
        content="""To configure AI Beast, edit the config/ai-beast.env file.
        Key settings include OLLAMA_HOST for the Ollama endpoint, DEFAULT_MODEL
        for the chat model, and EMBEDDING_MODEL for the embedding model. You can
        also set API keys for cloud providers like OPENAI_API_KEY and
        ANTHROPIC_API_KEY.""",
        metadata={"source": "configuration.md", "section": "environment"}
    )
]

print(f"Loaded {len(sample_docs)} documents")
for doc in sample_docs:
    print(f"  - {doc.metadata['source']}: {len(doc.content)} chars")

## 3. Chunking Documents

Large documents need to be split into manageable chunks.

In [ ]:
from modules.rag.chunker import Chunker

# Create chunker with different strategies
chunker = Chunker(
    strategy=ChunkingStrategy.FIXED_SIZE,
    chunk_size=200,      # Characters per chunk
    chunk_overlap=50,    # Overlap between chunks
)

# Chunk all documents
all_chunks = []
for doc in sample_docs:
    chunks = chunker.chunk(doc)
    all_chunks.extend(chunks)
    print(f"{doc.metadata['source']}: {len(chunks)} chunks")

print(f"\nTotal chunks: {len(all_chunks)}")

In [ ]:
# View a sample chunk
print("Sample chunk:")
print("-" * 50)
print(all_chunks[0].content)
print("-" * 50)
print(f"Metadata: {all_chunks[0].metadata}")

## 4. Creating Embeddings

Convert text chunks into vector embeddings for semantic search.

In [ ]:
from modules.rag.embeddings import EmbeddingClient

# Initialize embedding client
embedder = EmbeddingClient(
    model="nomic-embed-text"  # Or your preferred embedding model
)

# Generate embeddings for chunks
texts = [chunk.content for chunk in all_chunks]
embeddings = await embedder.embed_batch(texts)

print(f"Generated {len(embeddings)} embeddings")
print(f"Embedding dimension: {len(embeddings[0])}")

## 5. Vector Store

Store embeddings in Qdrant for efficient retrieval.

In [ ]:
from modules.rag.vectorstore import QdrantStore

# Initialize vector store
vectorstore = QdrantStore(
    collection_name="tutorial_docs",
    embedding_dim=len(embeddings[0]),
)

# Create collection (if needed)
await vectorstore.create_collection()

# Add documents with embeddings
await vectorstore.add_documents(
    documents=all_chunks,
    embeddings=embeddings,
)

print(f"Stored {len(all_chunks)} chunks in Qdrant")

## 6. Semantic Search

Search for relevant documents using natural language.

In [ ]:
# Search function
async def search(query: str, top_k: int = 3):
    """Search for relevant documents."""
    # Embed the query
    query_embedding = await embedder.embed(query)
    
    # Search vector store
    results = await vectorstore.search(
        query_embedding=query_embedding,
        top_k=top_k,
    )
    
    return results

# Test search
query = "How do I install AI Beast?"
results = await search(query)

print(f"Query: {query}\n")
print("Results:")
for i, result in enumerate(results, 1):
    print(f"\n{i}. Score: {result.score:.4f}")
    print(f"   Source: {result.document.metadata.get('source', 'unknown')}")
    print(f"   Content: {result.document.content[:100]}...")

## 7. Building the Q&A System

Combine retrieval with LLM for document Q&A.

In [ ]:
async def answer_question(question: str, top_k: int = 3) -> dict:
    """Answer a question using RAG."""
    
    # 1. Retrieve relevant context
    results = await search(question, top_k=top_k)
    
    # 2. Build context from results
    context_parts = []
    sources = []
    
    for result in results:
        context_parts.append(result.document.content)
        source = result.document.metadata.get('source', 'unknown')
        if source not in sources:
            sources.append(source)
    
    context = "\n\n".join(context_parts)
    
    # 3. Create prompt with context
    prompt = f"""Answer the question based on the context below. 
If the answer is not in the context, say "I don't have information about that."

Context:
{context}

Question: {question}

Answer:"""
    
    # 4. Generate answer
    response = await llm.chat(
        messages=[{"role": "user", "content": prompt}],
        options={"temperature": 0.3},
    )
    
    return {
        "question": question,
        "answer": response.content,
        "sources": sources,
        "num_chunks_used": len(results),
    }

# Test the Q&A system
result = await answer_question("What are the system requirements for AI Beast?")

print(f"Q: {result['question']}\n")
print(f"A: {result['answer']}\n")
print(f"Sources: {', '.join(result['sources'])}")

In [ ]:
# Test more questions
questions = [
    "What is Ollama?",
    "How do I configure AI Beast?",
    "What chunking strategies are available?",
    "What is the capital of France?",  # Not in docs
]

for q in questions:
    print("="*60)
    result = await answer_question(q)
    print(f"Q: {result['question']}")
    print(f"A: {result['answer'][:300]}..." if len(result['answer']) > 300 else f"A: {result['answer']}")
    print(f"Sources: {', '.join(result['sources'])}\n")

## 8. Using the RAG Pipeline

AI Beast provides a high-level RAG pipeline that handles all steps.

In [ ]:
from modules.rag import RAGPipeline, RAGConfig

# Initialize pipeline with config
rag = RAGPipeline(
    config=RAGConfig(
        collection_name="tutorial_pipeline",
        chunk_size=200,
        chunk_overlap=50,
        top_k=3,
        embedding_model="nomic-embed-text",
    )
)

# Ingest documents
await rag.ingest(sample_docs)
print("Documents ingested!")

In [ ]:
# Query using pipeline
result = await rag.query("What is AI Beast and what providers does it support?")

print("Answer:")
print(result.answer)
print(f"\nSources: {result.sources}")
print(f"Confidence: {result.confidence:.2f}")

## Next Steps

Excellent! You've built a document Q&A system with RAG.

Continue with:
- **[04_advanced_prompting.ipynb](04_advanced_prompting.ipynb)** - Master prompt engineering
- **[05_building_agents.ipynb](05_building_agents.ipynb)** - Create AI agents with tools

## Exercises

1. Load your own documents (PDFs, markdown, etc.)
2. Experiment with different chunking strategies
3. Add metadata filtering to search
4. Build a chat interface with persistent memory

## Tips

- **Chunk size matters**: Smaller chunks = more precise retrieval, larger = more context
- **Overlap helps**: Prevents losing information at chunk boundaries
- **Metadata is valuable**: Use it for filtering and source attribution
- **Embeddings choice**: Different models have different strengths